# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5940b971f0>
├── 'a' --> tensor([[ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718]])
└── 'x' --> <FastTreeValue 0x7f5940b97070>
    └── 'c' --> tensor([[-0.5901, -0.9554,  0.0833,  1.2324],
                        [-0.9830,  0.2653,  1.8226,  0.8970],
                        [ 1.3106, -1.3543,  0.4370,  0.2409]])

In [4]:
t.a

tensor([[ 0.9330, -1.8227,  0.5544],
        [ 0.5019, -1.2108, -0.3718]])

In [5]:
%timeit t.a

61.1 ns ± 0.0739 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5940b971f0>
├── 'a' --> tensor([[-0.0132, -1.1390,  0.7712],
│                   [ 1.4897,  0.0042, -1.8274]])
└── 'x' --> <FastTreeValue 0x7f5940b97070>
    └── 'c' --> tensor([[-0.5901, -0.9554,  0.0833,  1.2324],
                        [-0.9830,  0.2653,  1.8226,  0.8970],
                        [ 1.3106, -1.3543,  0.4370,  0.2409]])

In [7]:
%timeit t.a = new_value

73.3 ns ± 0.186 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718]]),
    x: Batch(
           c: tensor([[-0.5901, -0.9554,  0.0833,  1.2324],
                      [-0.9830,  0.2653,  1.8226,  0.8970],
                      [ 1.3106, -1.3543,  0.4370,  0.2409]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9330, -1.8227,  0.5544],
        [ 0.5019, -1.2108, -0.3718]])

In [11]:
%timeit b.a

52.6 ns ± 0.113 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7553,  0.5184,  0.7189],
               [-2.0457,  1.0813, -0.7820]]),
    x: Batch(
           c: tensor([[-0.5901, -0.9554,  0.0833,  1.2324],
                      [-0.9830,  0.2653,  1.8226,  0.8970],
                      [ 1.3106, -1.3543,  0.4370,  0.2409]]),
       ),
)

In [13]:
%timeit b.a = new_value

499 ns ± 0.171 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 2.15 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 75.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 337 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 511 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f58968a0580>
├── 'a' --> tensor([[[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]],
│           
│                   [[ 0.9330, -1.8227,  0.5544],
│                    [ 0.5019, -1.2108, -0.3718]]])
└── 'x' --> <FastTreeValue 0x7f58968a06a0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 64.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f59180b91c0>
├── 'a' --> tensor([[ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718],
│                   [ 0.9330, -1.8227,  0.5544],
│                   [ 0.5019, -1.2108, -0.3718]])
└── 'x' --> <FastTreeValue 0x7f5896fce8e0>
    └── 'c' --> tensor([[-0.5901, -0.9554,  0.0833,  1.2324],
                        [-0.9830,  0.2653,  1.8226,  0.8970],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 44.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.5 µs ± 80.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]],
       
               [[ 0.9330, -1.8227,  0.5544],
                [ 0.5019, -1.2108, -0.3718]]]),
    x: Batch(
           c: tensor([[[-0.5901, -0.9554,  0.0833,  1.2324],
                       [-0.9830,  0.2653,  1.8226,  0.8970],
                       [ 1.3106, -1.3543,  0.4370,  0.2409]],
         

In [26]:
%timeit Batch.stack(batches)

78.3 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718],
               [ 0.9330, -1.8227,  0.5544],
               [ 0.5019, -1.2108, -0.3718]]),
    x: Batch(
           c: tensor([[-0.5901, -0.9554,  0.0833,  1.2324],
                      [-0.9830,  0.2653,  1.8226,  0.8970],
                      [ 1.3106, -1.3543,  0.4370,  0.2409],
                      [-0.5901, -0.9554,  0.0833,  1.2324],
                      [-0.9830,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 317 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 1.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
